### Requirements

##### Install RISE slide show
- https://rise.readthedocs.io/en/latest/

##### Install Python packages
- pandas
- ipython

Install with the command: ```pip install <library>```

##### Run docker daemon on your computer


In [ ]:
import requests
import os
import json
import pandas as pd
from IPython.display import Markdown as md

In [ ]:
def view_json(file):
    with open(file, 'r') as json_file:
        json_object = json.load(json_file)
    print(json.dumps(json_object, indent=2))

In [ ]:
input_folder = '../input-examples/omop-fhir-data/synthea-random-020/'
output_folder = '../docker-compose/convert/volume/output/'
whistle_mappings_folder = '../whistle-mappings/synthea/'
validate_folder = '../docker-compose/validate/'

### Setup Dockers and Data

In [ ]:
%%bash
../docker-compose/bin/all-down.sh

In [ ]:
%%bash
../docker-compose/bin/all-build.sh >/dev/null 2>&1 &

In [ ]:
%%bash
../docker-compose/bin/hapi-up.sh >/dev/null 2>&1 &

In [ ]:
%%bash
../docker-compose/bin/omop-fhir-data-update.sh >/dev/null 2>&1 &

In [ ]:
%%bash
docker ps -a

### Start the RISE slideshow on the next slide
- SPACE BAR = next slide
- SHIFT + SPACE BAR = previous slide
- SHIFT + ENTER = Run cell

# OMOP-to-BULK FHIR: A tool to convert population level clinical data into standardized FHIR batch data


**Andrey Soares, PhD**  
Department of Medicine, University of Colorado Anschutz Medical Campus  

**Shahim Essaid, MD**  
Department of Biomedical Informatics, University of Colorado Anschutz Medical Campus  

**Michael G. Kahn, MD**  
Department of Biomedical Informatics, University of Colorado Anschutz Medical Campus  



##### Source: https://github.com/CU-DBMI/mends-on-fhir

### MENDS-on-FHIR
![Overall Design](../assets/images/MENDS-generalized.png)
* Multi-state EHR-based Network for Disease Surveillance (MENDS)

### View OMOP Source Data

In [ ]:
view_json(input_folder + 'Person_0000000000.json')

In [ ]:
view_json(input_folder + 'Condition_Occurrence_0000000000.json')

In [ ]:
view_json(input_folder + 'Measurement_0000000000.json')

In [ ]:
view_json(input_folder + 'Drug_Exposure_0000000000.json')

### Sample Concept Map

In [ ]:
view_json(whistle_mappings_folder + 'concept-maps/Person.ethnicity.conceptid.json') 

### Sample Whistle Transformation

In [ ]:
f = open(whistle_mappings_folder + '/whistle-functions/Drug_Exposure.wstl', "r")
print(f.read())

In [ ]:
f = open(whistle_mappings_folder + '/whistle-functions/Observation_Smoking.wstl', "r")
print(f.read())

### Convert OMOP Data to FHIR Resources

In [ ]:
%%bash
../docker-compose/bin/convert-up.sh >/dev/null 2>&1 &

### View FHIR Resource

In [ ]:
view_json(output_folder + 'Person_0000000000.output.json')

In [ ]:
view_json(output_folder + 'Condition_Occurrence_0000000000.output.json')

In [ ]:
view_json(output_folder + 'Visit_Occurrence_0000000000.output.json')

In [ ]:
view_json(output_folder + 'Measurement_0000000000.output.json')

In [ ]:
view_json(output_folder + 'Drug_Exposure_0000000000.output.json')

### Validate FHIR Resources

In [ ]:
%%bash
../docker-compose/bin/validate-up.sh

In [ ]:

folder = validate_folder + 'volume/results'
for file in sorted(os.listdir(folder), reverse=True):
    if file.endswith(".html"):
        filename = os.path.join(folder, file)
md("## View [Validation Result](%s)"%(filename))

### Count Condition Resources Before Load

In [ ]:
try:
    response = requests.get(url = 'http://localhost:8080/fhir/Condition')
    print(response.text)
except:
    pass

### Load FHIR Resources to HAPI FHIR Server

In [ ]:
%%bash
../docker-compose/bin/load-up.sh

### Retrieving Condition #1 in HAPI FHIR Server

In [ ]:
try:
    response = requests.get(url = 'http://localhost:8080/fhir/Condition/1')
    print(response.text)
except:
    pass

### Searching All Conditions

In [ ]:
try:
    response = requests.get(url = 'http://localhost:8080/fhir/Condition')
    print(response.text)
except:
    pass

In [ ]:
md("## Open <a href=\"http://localhost:8080\">HAPI FHIR Server</a>")